In [22]:
import ftplib
import gzip
import pandas as pd
from io import BytesIO

# FTP 연결 설정
ftp_host = 'varodrt.synology.me'
ftp_port = 21
ftp_user = '비밀'
ftp_password = '비밀' #암호화

# 서버 접속
ftp = ftplib.FTP()
ftp.connect(ftp_host, ftp_port)
ftp.encoding = 'utf-8'
ftp.sendcmd('OPTS UTF8 ON')  # 한글 사용을 위해 필수로 입력할 것
ftp.login(user=ftp_user, passwd=ftp_password)

# 디렉토리 설정 및 파일 목록 가져오기
ftp.cwd('/03. DATA Dam/전북 고속시외버스 DTG데이터(2024.6.1~2024.6.30)/압축파일')
file_list = ftp.nlst()  # 모든 파일 목록을 가져옴
gz_files = [file.strip() for file in file_list if file.endswith('.gz')]  # .gz 파일만 필터링

print(f"Files found: {gz_files}")  # 파일 목록 출력


Files found: ['part-r-00000.gz', 'part-r-00001.gz', 'part-r-00002.gz', 'part-r-00003.gz', 'part-r-00004.gz', 'part-r-00005.gz', 'part-r-00006.gz', 'part-r-00007.gz', 'part-r-00008.gz', 'part-r-00009.gz', 'part-r-00010.gz', 'part-r-00011.gz', 'part-r-00012.gz', 'part-r-00013.gz', 'part-r-00014.gz', 'part-r-00015.gz', 'part-r-00016.gz', 'part-r-00017.gz', 'part-r-00018.gz', 'part-r-00019.gz', 'part-r-00020.gz', 'part-r-00021.gz', 'part-r-00022.gz', 'part-r-00023.gz', 'part-r-00024.gz', 'part-r-00025.gz', 'part-r-00026.gz', 'part-r-00027.gz', 'part-r-00028.gz', 'part-r-00029.gz', 'part-r-00030.gz', 'part-r-00031.gz', 'part-r-00032.gz', 'part-r-00033.gz', 'part-r-00034.gz', 'part-r-00035.gz', 'part-r-00036.gz', 'part-r-00037.gz', 'part-r-00038.gz', 'part-r-00039.gz', 'part-r-00040.gz', 'part-r-00041.gz', 'part-r-00042.gz', 'part-r-00043.gz', 'part-r-00044.gz', 'part-r-00045.gz', 'part-r-00046.gz', 'part-r-00047.gz', 'part-r-00048.gz', 'part-r-00049.gz', 'part-r-00050.gz', 'part-r-00051.gz'

In [13]:
import ftplib
import gzip
import pandas as pd
from io import BytesIO

# FTP 연결 설정
ftp_host = 'varodrt.synology.me'
ftp_port = 21
ftp_user = '비밀'
ftp_password = '비밀' #암호화

# 서버 접속
ftp = ftplib.FTP()
ftp.connect(ftp_host, ftp_port)
ftp.encoding = 'utf-8'
ftp.sendcmd('OPTS UTF8 ON')  # 한글 사용을 위해 필수로 입력할 것
ftp.login(user=ftp_user, passwd=ftp_password)

# 디렉토리 설정 및 파일 목록 가져오기
ftp.cwd('/03. DATA Dam/전북 고속시외버스 DTG데이터(2024.6.1~2024.6.30)/압축파일')
file_list = ftp.nlst()  # 모든 파일 목록을 가져옴
gz_files = [file.strip() for file in file_list if file.endswith('.gz')]  # .gz 파일만 필터링

print(f"Files found: {gz_files}")  # 파일 목록 출력

# 위도와 경도의 이상치를 제거하는 함수
def remove_lat_lon_outliers(df):
    df_cleaned = df[(df['GPS Y'] >= -90) & (df['GPS Y'] <= 90) &  # 위도 범위
                    (df['GPS X'] >= -180) & (df['GPS X'] <= 180)]  # 경도 범위
    return df_cleaned

# 데이터 필터링 및 처리 함수
def process_gzip_file(ftp, file_name):
    all_chunks = []  # 각 청크를 담을 리스트

    # FTP 작업 시 현재 디렉토리 재설정
    try:
        ftp.cwd('/03. DATA Dam/전북 고속시외버스 DTG데이터(2024.6.1~2024.6.30)/압축파일')
        with BytesIO() as bio:
            # 파일 명을 출력해서 확인
            print(f"Attempting to retrieve file: {file_name}")

            try:
                ftp.retrbinary(f"RETR {file_name}", bio.write)  # 파일을 메모리로 읽음
            except ftplib.error_perm as e:
                print(f"FTP error while retrieving {file_name}: {e}. Skipping file.")
                return

            bio.seek(0)
            with gzip.open(bio, 'rt', encoding='utf-8') as f:
                # 청크 단위로 데이터 읽기 (메모리 절약)
                for chunk in pd.read_csv(f, delimiter='|', header=None, chunksize=1000000):
                    # 컬럼명 설정
                    columns = [
                        'DTG 모델명', '차대번호', '업종', '차량번호', '사업자등록번호', '운수회사소재지코드',
                        '운수회사코드', '일일주행거리', '누적주행거리', '운행일시', '속도', 'RPM',
                        '브레이크신호', 'GPS X', 'GPS Y', '방위각', '가속도 X', '가속도 Y', '통신상태코드', '운행지역코드'
                    ]
                    chunk.columns = columns

                    # 1. 차량번호가 '전북'으로 시작하는 데이터만 필터링
                    chunk = chunk[chunk['차량번호'].str.startswith('전북')]

                    # 2. 위도와 경도의 이상치 제거
                    chunk = remove_lat_lon_outliers(chunk)

                    # 3. 빈 데이터 프레임은 처리하지 않음
                    if chunk.empty:
                        continue

                    # 4. 안전한 날짜 변환 처리
                    chunk['운행일시'] = chunk['운행일시'].astype(str).str.slice(stop=-2)
                    chunk.loc[:, '운행일시'] = pd.to_datetime(chunk['운행일시'], format='%Y%m%d%H%M%S', errors='coerce')
                    chunk.loc[:, '날짜'] = chunk['운행일시'].dt.strftime('%Y-%m-%d')

                    # 처리된 청크를 리스트에 추가
                    all_chunks.append(chunk)

    except OSError as e:
        print(f"Error while decompressing file {file_name}: {e}")
        return
    except ftplib.all_errors as e:  # General FTP error handling
        print(f"FTP error: {e}. Skipping file: {file_name}")
        return

    # 하나의 데이터프레임으로 합침
    if all_chunks:
        final_df = pd.concat(all_chunks)
        output_filename = f"{file_name.replace('.gz', '')}_processed.csv"
        upload_to_nas(ftp, final_df, output_filename)

# FTP를 통해 처리된 CSV 파일을 NAS에 업로드하는 함수
def upload_to_nas(ftp, df, output_filename):
    if df.empty:
        print(f"No data to upload for {output_filename}. Skipping upload.")
        return

    with BytesIO() as bio:
        df.to_csv(bio, index=False, encoding='cp949')  # DataFrame을 CSV로 변환하여 메모리에 저장, 인코딩은 cp949
        bio.seek(0)
        try:
            ftp.cwd('/03. DATA Dam/전북 고속시외버스 DTG데이터(2024.6.1~2024.6.30)/처리된파일')  # 업로드할 디렉토리 경로로 이동
            ftp.storbinary(f"STOR {output_filename}", bio)
            print(f"Successfully uploaded {output_filename} to NAS")
        except ftplib.all_errors as e:  # Handle all possible FTP errors
            print(f"FTP error while uploading {output_filename}: {e}")

# .gz 파일 처리
for gz_file in gz_files:
    print(f'Processing file: {gz_file}')
    process_gzip_file(ftp, gz_file)

# FTP 연결 종료
ftp.quit()
print("FTP connection closed.")


Files found: ['part-r-00000.gz', 'part-r-00001.gz', 'part-r-00002.gz', 'part-r-00003.gz', 'part-r-00004.gz', 'part-r-00005.gz', 'part-r-00006.gz', 'part-r-00007.gz', 'part-r-00008.gz', 'part-r-00009.gz', 'part-r-00010.gz', 'part-r-00011.gz', 'part-r-00012.gz', 'part-r-00013.gz', 'part-r-00014.gz', 'part-r-00015.gz', 'part-r-00016.gz', 'part-r-00017.gz', 'part-r-00018.gz', 'part-r-00019.gz', 'part-r-00020.gz', 'part-r-00021.gz', 'part-r-00022.gz', 'part-r-00023.gz', 'part-r-00024.gz', 'part-r-00025.gz', 'part-r-00026.gz', 'part-r-00027.gz', 'part-r-00028.gz', 'part-r-00029.gz', 'part-r-00030.gz', 'part-r-00031.gz', 'part-r-00032.gz', 'part-r-00033.gz', 'part-r-00034.gz', 'part-r-00035.gz', 'part-r-00036.gz', 'part-r-00037.gz', 'part-r-00038.gz', 'part-r-00039.gz', 'part-r-00040.gz', 'part-r-00041.gz', 'part-r-00042.gz', 'part-r-00043.gz', 'part-r-00044.gz', 'part-r-00045.gz', 'part-r-00046.gz', 'part-r-00047.gz', 'part-r-00048.gz', 'part-r-00049.gz', 'part-r-00050.gz', 'part-r-00051.gz'

KeyboardInterrupt: 

## 38번째 gz파일에서 decompressing data 이슈 발생하여 아래와 같이 코드를 수정함

In [28]:
import ftplib
import gzip
import pandas as pd
from io import BytesIO

# FTP 연결 설정
ftp_host = 'varodrt.synology.me'
ftp_port = 21
ftp_user = '비밀'
ftp_password = '비밀' #암호화

# 서버 접속
ftp = ftplib.FTP()
ftp.connect(ftp_host, ftp_port)
ftp.encoding = 'utf-8'
ftp.sendcmd('OPTS UTF8 ON')  # 한글 사용을 위해 필수로 입력할 것
ftp.login(user=ftp_user, passwd=ftp_password)

# 디렉토리 설정 및 파일 목록 가져오기
ftp.cwd('/03. DATA Dam/전북 고속시외버스 DTG데이터(2024.6.1~2024.6.30)/압축파일')
file_list = ftp.nlst()  # 모든 파일 목록을 가져옴
gz_files = [file.strip() for file in file_list if file.endswith('.gz')]  # .gz 파일만 필터링

# 시작점 설정: 39번째 파일부터 처리
gz_files = gz_files[39:]  # 39번째 파일 재검토

print(f"Files found: {gz_files}")  # 파일 목록 출력

# 위도와 경도의 이상치를 제거하는 함수
def remove_lat_lon_outliers(df):
    df_cleaned = df[(df['GPS Y'] >= -90) & (df['GPS Y'] <= 90) &  # 위도 범위
                    (df['GPS X'] >= -180) & (df['GPS X'] <= 180)]  # 경도 범위
    return df_cleaned

# 데이터 필터링 및 처리 함수
def process_gzip_file(ftp, file_name):
    all_chunks = []  # 각 청크를 담을 리스트

    # FTP 작업 시 현재 디렉토리 재설정
    try:
        ftp.cwd('/03. DATA Dam/전북 고속시외버스 DTG데이터(2024.6.1~2024.6.30)/압축파일')
        with BytesIO() as bio:
            # 파일 명을 출력해서 확인
            print(f"Attempting to retrieve file: {file_name}")

            try:
                ftp.retrbinary(f"RETR {file_name}", bio.write)  # 파일을 메모리로 읽음
            except ftplib.error_perm as e:
                print(f"FTP error while retrieving {file_name}: {e}. Skipping file.")
                return

            bio.seek(0)
            with gzip.open(bio, 'rt', encoding='utf-8') as f:
                # 청크 단위로 데이터 읽기 (메모리 절약)
                for chunk in pd.read_csv(f, delimiter='|', header=None, chunksize=1000000):
                    # 컬럼명 설정
                    columns = [
                        'DTG 모델명', '차대번호', '업종', '차량번호', '사업자등록번호', '운수회사소재지코드',
                        '운수회사코드', '일일주행거리', '누적주행거리', '운행일시', '속도', 'RPM',
                        '브레이크신호', 'GPS X', 'GPS Y', '방위각', '가속도 X', '가속도 Y', '통신상태코드', '운행지역코드'
                    ]
                    chunk.columns = columns

                    # 1. 차량번호가 '전북'으로 시작하는 데이터만 필터링
                    chunk = chunk[chunk['차량번호'].str.startswith('전북')]

                    # 2. 위도와 경도의 이상치 제거
                    chunk = remove_lat_lon_outliers(chunk)

                    # 3. 빈 데이터 프레임은 처리하지 않음
                    if chunk.empty:
                        continue

                    # 4. 안전한 날짜 변환 처리
                    chunk['운행일시'] = chunk['운행일시'].astype(str).str.slice(stop=-2)
                    chunk.loc[:, '운행일시'] = pd.to_datetime(chunk['운행일시'], format='%Y%m%d%H%M%S', errors='coerce')
                    chunk.loc[:, '날짜'] = chunk['운행일시'].dt.strftime('%Y-%m-%d')

                    # 처리된 청크를 리스트에 추가
                    all_chunks.append(chunk)

    except OSError as e:
        print(f"Error while decompressing file {file_name}: {e}")
        return
    except ftplib.all_errors as e:  # General FTP error handling
        print(f"FTP error: {e}. Skipping file: {file_name}")
        return

    # 하나의 데이터프레임으로 합침
    if all_chunks:
        final_df = pd.concat(all_chunks)
        output_filename = f"{file_name.replace('.gz', '')}_processed.csv"
        upload_to_nas(ftp, final_df, output_filename)

# FTP를 통해 처리된 CSV 파일을 NAS에 업로드하는 함수
def upload_to_nas(ftp, df, output_filename):
    if df.empty:
        print(f"No data to upload for {output_filename}. Skipping upload.")
        return

    with BytesIO() as bio:
        df.to_csv(bio, index=False, encoding='cp949')  # DataFrame을 CSV로 변환하여 메모리에 저장, 인코딩은 cp949
        bio.seek(0)
        try:
            ftp.cwd('/03. DATA Dam/전북 고속시외버스 DTG데이터(2024.6.1~2024.6.30)/처리된파일')  # 업로드할 디렉토리 경로로 이동
            ftp.storbinary(f"STOR {output_filename}", bio)
            print(f"Successfully uploaded {output_filename} to NAS")
        except ftplib.all_errors as e:  # Handle all possible FTP errors
            print(f"FTP error while uploading {output_filename}: {e}")

# .gz 파일 처리
for gz_file in gz_files:
    print(f'Processing file: {gz_file}')
    process_gzip_file(ftp, gz_file)

# FTP 연결 종료
ftp.quit()
print("FTP connection closed.")


Files found: ['part-r-00042.gz', 'part-r-00043.gz']
Processing file: part-r-00042.gz
Attempting to retrieve file: part-r-00042.gz


ParserError: Too many columns specified: expected 20 and found 16

## Many to one csv

In [35]:
import ftplib
import pandas as pd
from io import BytesIO

# FTP connection settings
ftp_host = 'varodrt.synology.me'
ftp_port = 21
ftp_user = '비밀'
ftp_password = '비밀' #암호화
# Connect to FTP server
ftp = ftplib.FTP()
ftp.connect(ftp_host, ftp_port)
ftp.encoding = 'utf-8'
ftp.sendcmd('OPTS UTF8 ON')  # Allow UTF-8 encoding for handling Korean characters
ftp.login(user=ftp_user, passwd=ftp_password)

# Navigate to the directory with processed CSV files (Korean path)
ftp.cwd('/03. DATA Dam/전북 고속시외버스 DTG데이터(2024.6.1~2024.6.30)/처리된파일')

# Get a list of all .csv files in the directory
csv_files = [file for file in ftp.nlst() if file.endswith('.csv')]

# Initialize an empty list to store DataFrames
df_list = []

# Function to read a CSV file from the FTP into a DataFrame
def read_csv_from_ftp(file_name):
    with BytesIO() as bio:
        ftp.retrbinary(f"RETR {file_name}", bio.write)  # Download file into memory
        bio.seek(0)
        # Read CSV file into DataFrame using the likely encoding `cp949`
        df = pd.read_csv(bio, encoding='cp949')  # Changed from 'utf-8' to 'cp949'
        return df

# Loop through each .csv file and append its DataFrame to the list
for csv_file in csv_files:
    try:
        df = read_csv_from_ftp(csv_file)
        df_list.append(df)
        print(f"Successfully read {csv_file}")
    except Exception as e:
        print(f"Error reading {csv_file}: {e}")

# Concatenate all DataFrames into one
if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)
    print(f"Successfully concatenated {len(df_list)} files.")

    # Save the combined DataFrame into a single CSV file
    output_filename = 'combined_data.csv'
    with BytesIO() as bio:
        combined_df.to_csv(bio, index=False, encoding='cp949')  # Save DataFrame as CSV to memory with 'cp949' encoding
        bio.seek(0)
        # Upload the concatenated file back to FTP
        ftp.storbinary(f"STOR {output_filename}", bio)
        print(f"Successfully uploaded {output_filename} to FTP.")
else:
    print("No data to concatenate.")

# Close the FTP connection
ftp.quit()
print("FTP connection closed.")


Successfully read part-r-00000_processed.csv
Successfully read part-r-00001_processed.csv
Successfully read part-r-00004_processed.csv
Successfully read part-r-00007_processed.csv
Successfully read part-r-00008_processed.csv
Successfully read part-r-00010_processed.csv
Successfully read part-r-00011_processed.csv
Successfully read part-r-00012_processed.csv
Successfully read part-r-00014_processed.csv
Successfully read part-r-00015_processed.csv
Successfully read part-r-00016_processed.csv
Successfully read part-r-00021_processed.csv
Successfully read part-r-00024_processed.csv
Successfully read part-r-00025_processed.csv
Successfully read part-r-00026_processed.csv
Successfully read part-r-00028_processed.csv
Successfully read part-r-00033_processed.csv
Successfully read part-r-00034_processed.csv
Successfully read part-r-00035_processed.csv
Successfully read part-r-00036_processed.csv
Successfully read part-r-00037_processed.csv
Successfully read part-r-00039_processed.csv
Successful

## Many to 1milion rows

In [37]:
import ftplib
import pandas as pd
from io import BytesIO

# FTP connection settings
ftp_host = 'varodrt.synology.me'
ftp_port = 21
ftp_user = '비밀'
ftp_password = '비밀' #암호화

# Connect to FTP server
ftp = ftplib.FTP()
ftp.connect(ftp_host, ftp_port)
ftp.encoding = 'utf-8'
ftp.sendcmd('OPTS UTF8 ON')  # Allow UTF-8 encoding for handling Korean characters
ftp.login(user=ftp_user, passwd=ftp_password)

# Navigate to the directory with processed CSV files (Korean path)
ftp.cwd('/03. DATA Dam/전북 고속시외버스 DTG데이터(2024.6.1~2024.6.30)/처리된파일')

# Get a list of all .csv files in the directory
csv_files = [file for file in ftp.nlst() if file.endswith('.csv')]

# Initialize an empty list to store DataFrames
df_list = []

# Function to read a CSV file from the FTP into a DataFrame
def read_csv_from_ftp(file_name):
    with BytesIO() as bio:
        ftp.retrbinary(f"RETR {file_name}", bio.write)  # Download file into memory
        bio.seek(0)
        # Read CSV file into DataFrame using the likely encoding `cp949`
        df = pd.read_csv(bio, encoding='cp949')  # Changed from 'utf-8' to 'cp949'
        return df

# Loop through each .csv file and append its DataFrame to the list
for csv_file in csv_files:
    try:
        df = read_csv_from_ftp(csv_file)
        df_list.append(df)
        print(f"Successfully read {csv_file}")
    except Exception as e:
        print(f"Error reading {csv_file}: {e}")

# Concatenate all DataFrames into one
if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)
    print(f"Successfully concatenated {len(df_list)} files.")

    # Split and save the DataFrame into multiple CSV files, each with a max of 1 million rows
    row_limit = 1000000  # 1 million rows per file
    num_chunks = (len(combined_df) // row_limit) + 1  # Calculate how many files we need to create

    for i in range(num_chunks):
        start_row = i * row_limit
        end_row = min(start_row + row_limit, len(combined_df))
        chunk_df = combined_df.iloc[start_row:end_row]

        # Create filename for each chunk
        output_filename = f'combined_data_part_{i+1}.csv'

        with BytesIO() as bio:
            chunk_df.to_csv(bio, index=False, encoding='cp949')  # Save DataFrame as CSV to memory with 'cp949' encoding
            bio.seek(0)
            # Upload the chunked file back to FTP
            ftp.storbinary(f"STOR {output_filename}", bio)
            print(f"Successfully uploaded {output_filename} to FTP.")
else:
    print("No data to concatenate.")

# Close the FTP connection
ftp.quit()
print("FTP connection closed.")

# 문제가 있었던 파일 목록 출력
if failed_files:
    print("\nThe following files encountered errors and were skipped:")
    for failed_file in failed_files:
        print(f"- {failed_file}")


Successfully read part-r-00000_processed.csv
Successfully read part-r-00001_processed.csv
Successfully read part-r-00004_processed.csv
Successfully read part-r-00007_processed.csv
Successfully read part-r-00008_processed.csv
Successfully read part-r-00010_processed.csv
Successfully read part-r-00011_processed.csv
Successfully read part-r-00012_processed.csv
Successfully read part-r-00014_processed.csv
Successfully read part-r-00015_processed.csv
Successfully read part-r-00016_processed.csv
Successfully read part-r-00021_processed.csv
Successfully read part-r-00024_processed.csv
Successfully read part-r-00025_processed.csv
Successfully read part-r-00026_processed.csv
Successfully read part-r-00028_processed.csv
Successfully read part-r-00033_processed.csv
Successfully read part-r-00034_processed.csv
Successfully read part-r-00035_processed.csv
Successfully read part-r-00036_processed.csv
Successfully read part-r-00037_processed.csv
Successfully read part-r-00039_processed.csv
Successful